# Recommender Engine
We will put everything together to implement a recommender engine. Our Recommender engine is hybrid of follwoing 5 sub engines:<br>
* Weighted User based recommender
* Item based recommender
* Desinger based recommender
* Most Popular recommender
* Expert recommender

<br>
First of all, we calculate the 25%,50%,75% percentile number of purchase, which is used to decide which combination of recommender engines are triggered. The decision tree below summarize the decision process(tid is the unique tag of product, browse is the browing record of user) and the technique used to surpress overspecialization
<br>

The left red arrows is user's familiarity of Pinkoi, the deeper it is , the more familiar hence the probability of overspecialization get higher. The right blues arrows are techniques to combat overspecilaization. The darker it is , the stonger its ability to supress overspecialization. Note that the parameter rand_n  vary in opposite direction. it neutralize the other two techniques to prevent the loss of preciseness
<img src='Intermediate/RecDT.png' width=1860 height=980>

To understand the branching criteria and the logic behind which recommender engine is triggered, we have to understand the core idea and covergae of each engine:
<br>
### Recommendation Engine

#### MostPopularRec:
Simply extract 6 most popular product pre-calculated in DataExploration. This is specially for first-come user without any records before

#### ExpertRec
This engine is for user who doesn't purchase any product before but has some browing records. Here we don't consider browing time(because I don't have data) so the subcategory that is interesting to user is simply depends on the number of it in browing records. For example, if the browing record is [a,a,b,c,d,h], then this user is interested in subcategory 'a'. We thus find the expert of this subcategory then recommend 6 products bought by the expert. The definition of expert here is the user who has the highest number purchase of this subcategory, which is pre-calculated in ExpertCalculation 

#### Userbased recommender:
The goal of this engie is to provide precise recommendation.It is based on the nearsest usres pre-calculated in UserSimilarityCalculation. It searches the prurchase records of top 2 nearest users and calculate find the k most similar products among these products.

#### Itembased recommender
The goal of this engine is the same as Userbased recommender. Instead of searching similar user, it searches the product in subcategory user is most intereested in. Subcategory decided, the similarity only rely on the style of product and the style preference of user. Hence we calculate the cosine similarity of user's style preference and style of product(Product Tag here) then get the top K nearest products.

#### Designer based recommender
This engine aims to address the overspecialization problem caused by the previous two engines. It aims to provide novel and seredipitious recommendation list. The engine search the designer user likes most based on purchase record. It randomly choose the product from the union of this designer and the most similar designer. We choose randomly product instead of calculating cosine similarity as in item based recommender is because the goal of deisnger recommender is to increase the novelty and serendipity. After several experiments I found that if we calculate the cosine similarity again, the recommendataionlist would become too specialize. The reason why this engine can increase novelty and serendipity is based on weaker similarity metircs used here. It use designer, which is a broader class, instead of compare the product one on one, which is very specialize.

Table below smmarize the property of each engine:
<img src='Intermediate/recsum.png' width=600 height=400 >
Now it's reasonable to understand the branching criteria in decisoin tree. As the tree go deeper, the scope of product discovered by user become broader. Consequently, we should reduce the number of product recommended by engine with stronger specialization(Userbased,Itembased here) and increase the lower one(Designer based) in case that the user has seen the recommended product before. **In conclusion, as the user buy more( be more familiar with our product), we should focus on more on overspecialization issue**



Even though I apply the design aforementioned, over-specialization still happended. Hence I apply following two tricks to further surpress over-specialization:
### Techniques to surpress overspecialization

#### Hybrid of engine:
This is simply implemented by deciding how many items an engine can return. If we let the engine with stronger specialization then the overall recommendation list get more specialized

#### parameter of randomness: rand_p
Instead of asking the recommendation engine to return k product, we ask the engine to return rand_p * k products and randomly choose k product within the set of rand_p * k. As rand_p get larger, it becomes more difficult to get the most similar products, which reduce overspecialization. This parameter get smaller as the degree of over specialization of overall combination of recommender engine get lower(i,e the last combination of engine has the smallest rand_p while the first has the largest one )

#### Linearly decrese search scope of ProductRec:
As the tree(stage) go deeper, the less product scope the recommender engine search. By doing so, it can reduce the probability of finding very similar data the user has seen before thus reduce overspecialization

In [1]:
import pandas as pd
import numpy as np
from random import shuffle
import time

In [2]:
#Load all data pre-calculate data 
Productdata=pd.read_csv("Intermediate/data_final.csv",sep=",",encoding='utf8',low_memory=False)
Userdata=pd.read_csv("Intermediate/SimUserData.csv",sep=",",encoding='utf8')
Designer_sim=pd.read_csv("Intermediate/design_sim.csv",sep=",",encoding='utf8')
Designer_data=pd.read_csv("Intermediate/designer_data_list.csv",sep=",",encoding='utf8',header=None)
Popularity=pd.read_csv("Intermediate/Popularity.csv",sep=",",encoding='utf8')
UserSim=pd.read_csv("Intermediate/Similarity_user.csv",sep=",",encoding='utf8')
Expert=pd.read_csv("Intermediate/expert.csv",sep=",",encoding='utf8')

In [3]:
#Set the index of Productdata by tid, which would be useful for later indexing
Productdata.index=Productdata['tid']

In [4]:
Productdata['Product Tag']=Productdata['Product Tag'].fillna(u'0')

In [5]:
# This function calculate the percentile of numeber of purchase
def PurchaseStat(X):
    l=[len(e.split(",")) for e in X]
    return [np.percentile(l,25),np.percentile(l,50),np.percentile(l,75)]

In [6]:
P_stat=PurchaseStat(Userdata['tid'])

In [7]:
P_stat

[3.0, 8.0, 12.0]

In [9]:
#Following are three helper functions, which aims to achieve the fast sparse array calculation
#dic_converter: convert list of string to dictionary. the fromkeys, for.. method has been proved to be the fatest method 
#dic_sim:calculate the cosine similarity of two dic objects
#dic_max: return the key with maximum values, randomly choose one if more than one key has maximum values
def dic_converter(x):
    if x is np.nan:
        x=u'0'
    L=x.split(",")
    D={}.fromkeys(L,0)
    
    for e in L:
        D[e]+=1.0
    return D 

def dic_sim(x,y):
    common=set(x.keys()).intersection(set(y.keys()))
    norm_x=np.sqrt(sum([e*e for e in x.values()]))
    norm_y=np.sqrt(sum([e*e for e in y.values()]))
    
    s=0.0
    for c in common:
        s+=x[c]*y[c]
    s/=(norm_x*norm_y)

    return s

def dic_max(x):
    maximum=max(x.values())
    d_max=[k for k,v in x.items() if v == maximum]
    
    if len(d_max)==1:
        return d_max[0]
    else:
        return np.random.choice(d_max,1)[0]

In [10]:
#Return six most popular products
def MostPopularRec(num):
    rec=[]
    idx=np.random.randint(0,6,num)
    for i in idx:
        tmp=Productdata[Productdata['subcategory']==Popularity['subcategory'][i]]
        tmp_s=sorted(tmp['Popularity score'],reverse=True)
        rec.append(tmp.ix[tmp[tmp['Popularity score']==tmp_s[0]].index[0],'tid'])
    return rec

In [11]:
#return the product purchased by expert of this subcategory
def ExpertRec(browse):

    expert_list=list(Expert['subcategory'])
    rec=[]
    exp=[]
    candidate=[]
    for record in set(browse):
        sub_cat=Productdata[Productdata['tid']==record]['subcategory'].values[0]
        #Not all categpries have an expert so we have to use a if statement
        if sub_cat in expert_list:
             exp.extend(Expert[Expert['subcategory']==sub_cat]['UID'].values[0].split(","))
    rec_num=0
    
    for ex in exp:
        candidate.extend(Userdata.iloc[int(ex)]['tid'].split(","))
    #if there is no expert in subcategory or the number of purchase of expert is less than six, then ask help from MostRec
    if len(candidate)<6:
        rec.extend(candidate)
        rec.extend(MostPopularRec(6-len(candidate)))
    else:
        shuffle(candidate)
        rec.extend(candidate[0:6])
    return rec

In [12]:
#The similarity calculator for Designer,Product and User
def SimCal(uid_target,cand_list,top_num,mode):
    #Used for designer and user
    if mode!='Product':
        Uc=Userdata.ix[uid_target,'subcategory']
        Us=Userdata.ix[uid_target,'Product Tag']

        Uc_dic=dic_converter(Uc)
        max_c=max(Uc_dic.values())
        for k in Uc_dic.keys():
            Uc_dic[k]=float(Uc_dic[k])/float(max_c)

        Us_dic=dic_converter(Us)
        
        #Calculating of the subcategory similarity
        Cat_dic=[dic_converter(e) for e in Productdata.ix[cand_list,'subcategory']]
        Cat_sim=[dic_sim(Uc_dic,e) for e in Cat_dic]
        #Calculation of the style similarity
        Style_dic=[dic_converter(e) for e in Productdata.ix[cand_list,'Product Tag']]
        Style_sim=[dic_sim(Us_dic,e) for e in Style_dic]
        
        score=list(np.array(Cat_sim)+np.array(Style_sim))
        score_uniqsort=sorted(list(set(score)),reverse=True)
        usr_rec_idx=[]
        kk=0
        while len(usr_rec_idx)<top_num:
            usr_rec_idx.extend([i for i,val in enumerate(score) if val==score_uniqsort[kk]])
            kk+=1
        return [e for e in cand_list if cand_list.index(e) in usr_rec_idx[0:top_num]]
    else:
        #This part is for similarity of product. When calculating product, we only have to condiser style similarity since the subcategory is all
        #the same
        Us=Userdata.ix[uid_target,'Product Tag']
        weight_s=Userdata.ix[uid_target,'Style weight']
        Us_dic=dic_converter(Us)
        
        
        Style_dic=[dic_converter(e) for e in Productdata.ix[cand_list,'Product Tag']]
        Style_sim=[dic_sim(Us_dic,e) for e in Style_dic]

        score_uniqsort=sorted(list(set(Style_sim)),reverse=True)
        pd_rec_idx=[]
        kk=0
        while len(pd_rec_idx)<top_num:
            pd_rec_idx.extend([i for i,val in enumerate(Style_sim) if val==score_uniqsort[kk]])
            kk+=1
        return [e for e in cand_list if cand_list.index(e) in pd_rec_idx[0:top_num]]       

In [13]:
def UserRec(usr,k):
    #filter out the product already seen by user
    rec=[]
    bought=usr['tid'].split(",")
    if usr['Browse']==u'0':
        seen=set(bought)
    else:
        browse=usr['Browse'].split(",")
        seen=set(bought).union(set(browse))
    
    #Extract the product list from 2 most similar user
    usr_sim=UserSim.iloc[usr['UID']]
    sim_item=[]
    for u in usr_sim:
        sim_item.extend(Userdata.ix[u,'tid'].split(","))

    candidate=[e for e in list(set(sim_item).difference(seen)) if e!=u'0']
    
    #if the number of candidate is smaller than 6 then ask help from MostPopularRec
    if len(candidate)<k:
        rec.extend(candidate)
        rec.extend(MostPopularRec(k-len(candidate)))
    else:
        rec.extend(SimCal(usr['UID'],candidate,k,'User'))
    return rec

In [14]:
def ProductRec(usr,k,rcl,stage):
    #filter out the product already seen by user
    rec=[]
    bought=usr['tid'].split(",")
    if usr['Browse']==u'0':
        seen=set(bought)
    else:
        browse=usr['Browse'].split(",")
        seen=set(bought).union(set(browse))
    
    #Put the products seen by user and the previous rec_list together to form invalid, which means 
    invalid=set(rcl).union(seen)
    Uc=usr['subcategory']
    
    Uc_dic=dic_converter(Uc)
    max_c=dic_max(Uc_dic)
    candidate=list(set(Productdata[Productdata['subcategory']==max_c]['tid']).difference(invalid))
    #I only consider 5000 products, It is used to reduce product space, trade of between response time and accuracy.
    #By doing so, it can also reduce overspecialization since we add randomness
    if len(candidate)>5000-stage*1000:
        shuffle(candidate)
        candidate_rnd=candidate[0:5000-stage*1000]
    else:
        candidate_rnd=candidate
        
    rec.extend(SimCal(usr['UID'],candidate_rnd,k,'Product'))
    return rec

In [15]:
def DesignerRec(usr,k,rcl):
    #Filter out the product seen by user
    rec=[]
    bought=usr['tid'].split(",")
    if usr['Browse']==u'0':
        seen=set(bought)
    else:
        browse=usr['Browse'].split(",")
        seen=set(bought).union(set(browse))
    
    invalid=set(rcl).union(seen)
    designer_dic=dic_converter(usr['designer'])
    
    #print designer_dic
    for dk in designer_dic.keys():
        if dk not in list(Designer_data[0]):
            del designer_dic[dk]
    #print designer_dic.keys()
    max_d=dic_max(designer_dic)
    
    #print max_d
    d_idx=Designer_data[Designer_data[0]==max_d].index[0]
    d_idxsim=Designer_sim.ix[d_idx,3]

    cand_desinger=list(Designer_data.ix[[d_idx,d_idxsim],0])
    cand=[]
    for d in cand_desinger:
        cand.extend(list(Productdata[Productdata['owner']==d]['tid']))
    candidate=list(set(cand).difference(invalid))

    rec.extend(np.random.choice(candidate,k,replace=False))
    return rec    

In [16]:
def Recommender(usr,rand_p):
    rec_list=[]
    if (usr['tid']==u'0') and (usr['Browse']==u'0'):
        print "Trigger MostPopularRec Engine"
        rec_list=MostPopularRec(6)
        
    elif (usr['tid']==u'0') and (len(usr['Browse'].split(","))>=1):
        print "Trigger ExpertRec Engine"
        rec_list=ExpertRec(usr['Browse'].split(","))
    else:
        purchase=len(usr['tid'].split(","))
        try:
            if purchase<P_stat[0]:
                print "Trigger stage 1"
                rec_list.extend(UserRec(usr,3))
                rec_list.extend(np.random.choice(ProductRec(usr,3*(rand_p),rec_list,1),3,replace=False))

            elif  P_stat[0]<purchase<P_stat[1]:
                print "Trigger stage 2"
                rec_list.extend(UserRec(usr,2))
                rec_list.extend(np.random.choice(ProductRec(usr,3*(rand_p-1),rec_list,2),3,replace=False))
                rec_list.extend(DesignerRec(usr,1,rec_list))

            elif P_stat[1]<purchase<P_stat[2]:
                print "Trigger stage 3"
                rec_list.extend(UserRec(usr,2))
                rec_list.extend(np.random.choice(ProductRec(usr,2*(rand_p-2),rec_list,3),2,replace=False))
                rec_list.extend(np.random.choice(DesignerRec(usr,2*(rand_p-2),rec_list),2,replace=False))

            else:
                print "Trigger stage 4"
                rec_list.extend(UserRec(usr,1))
                rec_list.extend(np.random.choice(ProductRec(usr,2*(rand_p-2),rec_list,4),2,replace=False))
                rec_list.extend(DesignerRec(usr,3,rec_list))
        except:
                print "Not enough recommend list due to some population is less than sample, call MostPopularRec for help "
                rec_list.extend(MostPopularRec(6-len(rec_list)))
    if len(rec_list)<6:
        rec_list.extend(MostPopularRec(6-len(rec_list)))

    return rec_list

# DEMO

In [ ]:
#Randomly choose an user
Usr_rnd=Userdata.ix[np.random.randint(0,len(Userdata),1),:]

#Print user info
print "tid"
print Userdata.ix[int(Usr_rnd['UID']),'tid']

print "Subcategory"
print Userdata.ix[int(Usr_rnd['UID']),'subcategory']
print "Product Tag"
print Userdata.ix[int(Usr_rnd['UID']),'Product Tag']

#Recommend product to user
start_time = time.time()
Rec_list=Recommender(Userdata.ix[int(Usr_rnd['UID']),:],9)
print "Respond time %s seconds" % (time.time() - start_time)

#Show product link
url=['https://www.pinkoi.com/product/'+e for e in Rec_list]
for u in url:
    print u